In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse, make_scorer
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

In [2]:
df_train = pd.read_csv("input/train.csv")
df_test = pd.read_csv("input/test.csv")

In [3]:
X = df_train.drop("price", axis=1)
y = df_train["price"]

In [4]:
sta = StandardScaler()
X = sta.fit_transform(X)

In [ ]:
model = GradientBoostingRegressor(criterion="mse")
params = {
        "n_estimators":[150, 200],
        "max_depth":[15, 20, 30],
        "max_features":[5,6,"auto"],
        "min_samples_split":[2, 3, 4],
        "min_samples_leaf":[1,2,3]
}


clf = GridSearchCV(
                estimator=model,
                param_grid=params
)

grid = clf.fit(X, y)



In [8]:
best = grid.best_estimator_
best

GradientBoostingRegressor(criterion='mse', max_depth=15, n_estimators=150)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=444)

In [10]:
y_pred_train = best.predict(X_train)
y_pred_test = best.predict(X_test)

In [11]:
df_y = pd.DataFrame(y_train)
df_y["y_pred"] = y_pred_train
df_y.head()

,price,y_pred
29816,9.395,9.394911
14427,7.210,7.196957
39546,9.321,9.322289
7814,6.964,6.956880
13427,7.671,7.672098


In [12]:
mse_train = mse(y_train, y_pred_train)
mse_test = mse(y_test, y_pred_test)

In [13]:
mse_train, mse_test

(0.0006671773564544866, 0.0006399585723932871)

In [14]:
score_mse = make_scorer(mse)
cross_validate(estimator=best, X=X, y=y, scoring=score_mse, cv=10)

KeyboardInterrupt: 

In [15]:
gbr_fit = best.fit(
                X=X,
                y=y)

In [16]:
df_test_sta = sta.fit_transform(df_test)

In [17]:
test_pred = gbr_fit.predict(df_test_sta)

In [18]:
test = pd.DataFrame(df_test.index, columns=["id"])
test["price"] = test_pred

In [19]:
test.head()

,id,price
0,0,7.227907
1,1,8.356155
2,2,7.850991
3,3,8.500437
4,4,9.293788


In [20]:
test.to_csv("GradientBoostingRegressor.csv", index = False, header = True)